# Importing libraries:

In [1]:
import os
import random
import scipy.io
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image


os provides a way to interact with the operating system, such as reading and writing files.

random is used to generate random numbers, which can be useful for shuffling indices or selecting random samples from a dataset.

scipy.io is used to load the labels from a .mat file.

torch is the main PyTorch library and provides many functions and classes for building and training deep learning models.

torchvision.transforms provides a set of image transformation functions, such as resizing, cropping, and normalizing the image data.

torchvision.models provides pre-trained deep learning models, such as ResNet and VGG.

torch.utils.data.Dataset is a PyTorch class that represents a dataset and provides methods for retrieving samples from it.

torch.utils.data.DataLoader is a PyTorch class that provides an iterator over a dataset, allowing you to easily iterate over batches of samples
.
sklearn.metrics provides various performance metrics for classification tasks, such as accuracy, precision, recall, and F1 score.

PIL.Image is a Python Imaging Library that provides functions for working with images, such as opening, resizing, and saving them.

# Defining the FlowerDataset class:

In [2]:
class FlowerDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, f'image_{idx + 1:05d}.jpg')
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx] - 1  # subtract 1 to make labels 0-indexed

        if self.transform:
            image = self.transform(image)

        return image, label

FlowerDataset is a custom PyTorch dataset class that represents the Flower Dataset.

__init__ is a method that initializes the dataset by setting the root directory, labels, and an optional transform.

__len__ is a method that returns the length of the dataset, which is the number of images.

__getitem__ is a method that retrieves a specific sample from the dataset given an index idx. It opens the corresponding image file, applies the specified transform (if any), and returns the image and label.

load_flower_data is a function that loads the Flower Dataset and creates a data loader for each split (training, validation, and test).

# Defining the load_flower_data function:

In [3]:
def load_flower_data(root_dir, mat_file, train_frac=0.8, random_seed=42):
    # Load labels from mat file
    labels = scipy.io.loadmat(mat_file)['labels'][0]

    # Split images into training and validation sets
    num_images = len(labels)
    indices = list(range(num_images))
    random.seed(random_seed)
    random.shuffle(indices)
    split_idx = int(train_frac * num_images)
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]

    # Define transformations to apply to the images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Create datasets and data loaders
    train_dataset = FlowerDataset(root_dir, labels[train_indices], transform)
    val_dataset = FlowerDataset(root_dir, labels[val_indices], transform)
    test_dataset = FlowerDataset(root_dir, labels, transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader

load_flower_data is a function that loads the Flower Dataset and creates data loaders for the training, validation, and test sets.

root_dir is the directory where the images are stored.

mat_file is the .mat file containing the labels for each image.

train_frac is the fraction of images to use for training (the rest are split between validation and test).

random_seed is the seed for the random number generator used for shuffling the indices.

labels are loaded from the .mat file and split into training and validation sets using the specified fraction and random seed.

transform is a series of image transformations to apply to the images, including resizing, cropping, converting to a tensor, and normalizing the pixel values.

train_dataset, val_dataset, and test_dataset are created using the FlowerDataset class, with the appropriate labels and transformations.

train_loader, val_loader, and test_loader are created using the DataLoader class, which provides an iterator over the datasets in batches of the specified size.

# Defining the evaluate function:

In [4]:
def evaluate(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return acc, precision, recall, f1

evaluate is a function that evaluates the performance of the model on a given data loader.

model is the trained deep learning model.

loader is the data loader containing the images and labels to evaluate.

model.eval() sets the model to evaluation mode, which disables dropout and batch normalization.

y_true and y_pred are empty lists that will be filled with the true and predicted labels, respectively.

torch.no_grad() disables gradient computation, which speeds up the evaluation and saves memory.

For each batch of images and labels in the loader, the images and labels are moved to the device (CPU or GPU), the model is used to predict the labels, and the true and predicted labels are added to the corresponding lists.

Several performance metrics are computed using the true and predicted labels, including accuracy, precision, recall, and F1 score.

The metrics are returned as a tuple.

# Setting up the device

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

device is set to 'cuda' if a CUDA-enabled GPU is available, otherwise it is set to 'cpu'.

# Loading the Flower Dataset:

In [6]:
root_dir = 'jpg'
mat_file = 'imagelabels.mat'
num_classes = len(set(scipy.io.loadmat(mat_file)['labels'][0]))
train_loader, val_loader, test_loader = load_flower_data(root_dir, mat_file)

The Flower Dataset is loaded using the load_flower_data function, with the root directory 'root_dir' and the .mat file containing the labels 'mat_file'.

The training, validation, and test data loaders are returned and assigned to train_loader, val_loader, and test_loader, respectively.

# Setting up the model:

In [7]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(2048, num_classes)
model = model.to(device)

C:\Users\mm\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mm\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The model is set up by loading a pre-trained ResNet50 model from models.resnet50(pretrained=True).

The model is moved to the device (CPU or GPU) for training.

# Defining the loss function and optimizer:

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

criterion is set to the cross-entropy loss function, which is commonly used for classification tasks.

optimizer is set to stochastic gradient descent (SGD) with a learning rate of 0.001 and momentum of 0.9.

# Training the model:

In [9]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        target = labels.type(torch.int64)  # Convert target tensor to int64
        loss = criterion(outputs, target)  # Use the updated target tensor
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    val_acc, val_precision, val_recall, val_f1 = evaluate(model, val_loader)
    print(
        f'Epoch {epoch + 1}/{num_epochs}: train_loss={train_loss:.4f}, val_acc={val_acc:.4f}, val_precision={val_precision:.4f}, val_recall={val_recall:.4f}, val_f1={val_f1:.4f}')

# Evaluate the model on the test set
test_acc, test_precision, test_recall, test_f1 = evaluate(model, test_loader)
print(
    f'Test Accuracy: {test_acc:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f},Test F1 Score: {test_f1:.4f}')


C:\Users\mm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10: train_loss=4.5701, val_acc=0.0159, val_precision=0.0017, val_recall=0.0159, val_f1=0.0030


C:\Users\mm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/10: train_loss=4.4590, val_acc=0.0256, val_precision=0.0033, val_recall=0.0256, val_f1=0.0051


C:\Users\mm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 8/10: train_loss=3.1509, val_acc=0.0098, val_precision=0.0066, val_recall=0.0098, val_f1=0.0066


C:\Users\mm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 9/10: train_loss=2.7467, val_acc=0.0140, val_precision=0.0115, val_recall=0.0140, val_f1=0.0101


C:\Users\mm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 10/10: train_loss=2.3127, val_acc=0.0128, val_precision=0.0074, val_recall=0.0128, val_f1=0.0081
Test Accuracy: 0.0151, Test Precision: 0.0123, Test Recall: 0.0151,Test F1 Score: 0.0119
